In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

!pip install keras

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import gc
import numpy as np
import pandas as pd
#import contractions
import re
#import matplotlib.pyplot as plt
#from tqdm.notebook import tqdm_notebook
#from nltk.corpus import stopwords
#from nltk.corpus import words
#from nltk.stem import WordNetLemmatizer
#import keras
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
#from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
#from tensorflow.keras.layers import Dense, Dropout, Input, Concatenate, Average, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
#from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras.layers import Input, Dense

In [ ]:
'''train_data =  pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
val_data =  pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv")
test_data = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv")
unintended_bias_train = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
'''

In [ ]:
#display(train_data["comment_text"][10])

In [ ]:
#[re.search("https?\S+", x) for x in train_data["comment_text"]]
#[re.findall("\w+.jpg\W*", x) for x in train_data["comment_text"]]
#[re.findall("@\S+\b", x) for x in train_data["comment_text"]]


In [ ]:
#train_data["comment_text"][0]

In [ ]:
 '''def preprocess(train_data): 
        train_data = train_data.apply(lambda x : contractions.fix(x, slang = False)) 
        train_data = train_data.apply(lambda x : x.lower()  )
        train_data = train_data.apply(lambda x : re.sub(r'[^\w\s]','',x)  )
        train_data = train_data.apply(lambda x : re.sub(r'https?\S+','', x)  )
        train_data = train_data.apply(lambda x : re.sub(r'\w+.jpg\W*','', x)  )
        train_data = train_data.apply(lambda x : re.sub(r'@\S+\b','',  x)  )
        train_data = train_data.apply(lambda x : re.sub(r'#\S+\b','', x)  )
        stop_words = set(stopwords.words("english"))
        #stop_words = set(stop_words) 
        train_data = train_data.apply(lambda x: re.sub("[@_!#$%^&*()<>?/|}{~:0-9]",'', x))
        train_data = train_data.apply(lambda x : re.sub(r'\s+',' ', x) )
        train_data = train_data.apply(lambda x: x.strip())
        train_data = train_data.apply(lambda x : [word for word in x.split() if word not in stop_words])
         
        #train_data= train_data.apply(lambda x: ' '.join(x))
        return train_data'''

In [ ]:
'''x = "explanation\nWhy sat"
contractions.fix(x, slang = False)
x = x.lower()
x = re.sub(r'[^\w\s]','',x)
x = re.sub(r'https?\S+','', x)
x = re.sub(r'\w+.jpg\W*','', x)
#re.sub(r'\w+.jpg\W*','',"explanation")
x = re.sub(r'@\S+\b','',  x)
x = re.sub(r'#\S+\b','', x) 
x = re.sub("[@_!#$%^&*()<>?/|}{~:0-9]",'', x)
x = re.sub(r'#\S+\b','', x)
x = re.sub(r'\s+',' ', x)
x = [word for word in x.split() if word not in stop_words]
x'''

In [ ]:
'''stop_words = set(stopwords.words("english"))
"why" in stop_words'''

In [ ]:
'''#remove all those special texts whivh won't be helpful 
train_data["comment_text"] = preprocess(train_data["comment_text"])'''

In [ ]:
'''train_data["comment_text"][0]'''

In [ ]:
''' def text_trunc(text):
    if len(text)>512:
        text =  text[:512]
    return text'''

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):

    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [ ]:
def regular_encode(texts, tokenizer, maxlen = 512 ):
    enc_di = tokenizer.batch_encode_plus(texts,return_attention_mask = False, return_token_type_ids  = False,
                                        pad_to_max_length = True, max_length = maxlen, truncation = True)
    return np.array(enc_di["input_ids"])
           
    
    

In [ ]:
import transformers

In [ ]:
def build_model(transformer,maxlen = 512):
    input_word_ids = Input(shape=(maxlen,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
import tensorflow as tf
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)


In [ ]:
from kaggle_datasets import KaggleDatasets
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 152
MODEL = 'jplu/tf-xlm-roberta-large'

In [ ]:
#from transformers import TFAutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
'''# Configuration
MODEL = 'jplu/tf-xlm-roberta-large'
AUTO = tf.data.experimental.AUTOTUNE
SEED = 2020
EPOCHS_1 = 20
EPOCHS_2 = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
SHUFFLE = 2048
VERBOSE = 1'''

In [ ]:
import pandas as pd

In [ ]:
train_data =  pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
val_data =  pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv")
test_data = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv")
unintended_bias_train = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")


In [ ]:
#unintended_bias_train.toxic = unintended_bias_train.toxic.round().astype(int)

In [ ]:
#unintended_bias_train.toxic.value_counts()

In [ ]:
# Combine train1 with a subset of train2
train = pd.concat([
    train_data[['comment_text', 'toxic']],
    unintended_bias_train[['comment_text', 'toxic']].query('toxic==1'),
    unintended_bias_train[['comment_text', 'toxic']].query('toxic==0').sample(n=50000, random_state=0)
])

In [ ]:
import numpy as np 


In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        #return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [ ]:
type(test_data.content.values)

In [ ]:
%%time 

x_train = regular_encode(list(train.comment_text.values), tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(list(val_data.comment_text.values), tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(list(test_data.content.values), tokenizer, maxlen=MAX_LEN)

#y_train = train_data.toxic.values
y_valid = val_data.toxic.values

In [ ]:
y_train = train.toxic.values
#y_valid = val_data.toxic.values

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [ ]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, maxlen=MAX_LEN)
model.summary()

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS
)

In [ ]:
 y_t = model.predict(test_dataset, verbose=1)

In [ ]:
s = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv")

In [ ]:
sub = pd.DataFrame({"id": s.id , "toxic":y_t.squeeze()}, index = None)

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
s